## ● FB  Place API進行地點資料的擷取 Ming Hsun

### 轉換fb圓形搜尋半徑，以矩形作為資料蒐尋空間範圍

In [ ]:
#主要使用的模組
import requests
import json
import csv

In [ ]:
#設定搜尋半徑
import math
radius=200
r=math.sqrt((radius**2)/2)
r

In [ ]:
#給定空間範圍左上、左下、右下座標值
lefup=[24.184761, 120.642457]
lefdown=[24.174197, 120.643269]
rightup=[24.184290, 120.647805]

In [ ]:
#在給定的座標範圍內，計算每一搜尋半徑200公尺的座標
lefup_lat=lefup[0]
lefdown_lat=lefdown[0]
m_lat_1deg=110574
dif_lat=1*(r/m_lat_1deg)
coordinate={}
temp_lat=lefup_lat
print dif_lat
while(temp_lat>lefdown_lat):
    coordinate.setdefault(temp_lat,[])
    temp_lat=temp_lat-dif_lat*2

In [ ]:
#將各個搜尋中心座標值存出為CSV檔
coor_list=[["x","y"]]
for lat in coordinate:
    for lng in coordinate[lat]:
        coor_list.append([lng,lat])
f=open("center.csv","w")
w = csv.writer(f) 
w.writerows(coor_list)
f.close()  

In [ ]:
for a in coordinate:
    print len(coordinate[a]),
#     print a,coordinate[a][0],coordinate[a][-1]
print len(coordinate)

In [ ]:
#FB token
token="EAACEdEose0cBALd3eoUBQmoBOgZBtwe7y0Ob4ZA6hxi91ZCMdCiQVmrNJ98K8TxYIoNVyVUjJOv1YJknsQn86SaV6cuyINASJ8ott7z1r156wJkev8yWNk3JXRqOumTGrjTh8Grpc5ZBKP5KnaZBlMftMcoTH9DXV7ZAjD2qYEgWDOuZAIHjXp20FDcfordeVsZD"

In [ ]:
placeinfo_list=[["place_id","name","x","y","checkins","street","phone","link","category","category_id",]]
for cor_lat in coordinate:
    lat=cor_lat
    for cor_lng in coordinate[cor_lat]:
        lng=cor_lng
        res=requests.get("https://graph.facebook.com/v2.9/search?type=place&center="+str(lat)+","+str(lng)+"&distance="+str(distance)+"&fields="+str(fields)+"&access_token="+token)
        placedata = json.loads(res.text)
        paging_count=0
        place_count=0
        while "paging" in placedata and 'next' in placedata['paging']:
            paging_count+=1
            place_count+=len(placedata['data'])
            for p in placedata["data"]:
                place_id=p["id"]
                place_name=p["name"].encode("utf8")
                x=p["location"]["longitude"]
                y=p["location"]["latitude"]
                checkins=p["checkins"]
                try:
                    street=p["location"]["street"].encode("utf8")
                except:
                    street=""
                try:
                    phone=p["phone"].encode("utf8")
                except:
                    phone=""
                try:
                    link=p["link"].encode("utf8")
                except:
                    link=""
                try:
                    category=[]
                    category_id=[]
                    for cat in p['category_list']:
                        category.append(cat["name"].encode("utf8"))
                        category_id.append(cat["id"])
                except:
                    category=""
                placeinfo_list.append([place_id,place_name,x,y,checkins,street,phone,link,category,category_id])
            n_ext=placedata['paging']['next']
            res=requests.get(n_ext)
            placedata = json.loads(res.text)
#         print "總頁數:",paging_count,
#         print "Place 總數:",place_count,
        print lng,lat,place_count
    time.sleep(120)
    print"------------------------------------------"
# f=open(str(lat)+"_"+str(lng)+"places.csv","w")
f=open("118fb_places.csv","w")
w = csv.writer(f) 
w.writerows(placeinfo_list)
f.close()   